# Incorporating micro-feedback effects

In [1]:
import sys
sys.path.append("../../")
import taxcalc
from taxcalc import *
import pandas as pd
import numpy as np

### Start by creating a Calculator for Plan X and a calculator for Plan Y. 

In [2]:
# Create a Records object for Plan X and Plan Y
records_x = Records("../../puf.csv")
records_y = Records("../../puf.csv")
# Create a Parameters object for Plan X and Plan Y
params_x = Parameters()
params_y = Parameters()

# Create two Calculators
calcX = Calculator(params_x, records_x)
calcY = Calculator(params_y, records_y)


You loaded data for 2008.
Your data have beeen extrapolated to 2013.
You loaded data for 2008.
Your data have beeen extrapolated to 2013.


### Increase the top marginal tax rate by 10 percentage points 

In [3]:
# Modify the relevant parameter for the Plan-Y Calculator.
reform = {2013: {"_II_rt7": [0.496]}}
params_y.implement_reform(reform)
# Demonstrate that Plan X and Plan Y calculators are indeed different. 
print(calcX.params.II_rt7)
print(calcY.params.II_rt7)

0.396
0.496


### Update taxpayers' income to account for the rate hike with our behavioral effects calculator

In [4]:
# Call the behavioral effects calculator and create a new Plan Y Calculator obect. 
calcY_behavioral = behavior(calcX, calcY)
# Demonstrate that taxpayers' income was affected by the tax change.
print(calcY.records.e00200.sum())
print(calcY_behavioral.records.e00200.sum())

AttributeError: 'Calculator' object has no attribute '_ospctax'